In [1]:
import pandas as pd

## 레진 데이터 챌린지 2017

오늘의 주제는 [레진 데이터 챌린지 2017](http://tech.lezhin.com/events/data-challenge-pyconkr-2017)의 데이터를 바탕으로, 레진코믹스의 구매량과 주 사용 플랫폼과 작품 등의 상관관계를 분석하는 것입니다. 총 데이터는 650,965 건이며, 이 데이터를 판다스로 활용해 분석할 수 있어야 합니다.

[원래 경진대회](http://tech.lezhin.com/events/data-challenge-pyconkr-2017) 에서는 머신러닝을 활용해 구매 예측 여부를 판단하였지만, 이번 과제에서는 머신러닝을 사용하지 않고, 판다스만을 사용해 분석할 것입니다.

데이터는 [다음의 링크](https://storage.googleapis.com/lz-insight/pycon17/dataset/lezhin_dataset_v2_training.tsv.gz)에서 다운받으실 수 있습니다.

** 목표 **

주어진 데이터를 활용하여, 타 팀에서 요청한 10여개의 질문에 대한 답변을 줘야 합니다. 툴은 파이썬과 판다스를 사용하며, 타 팀에서 직관적으로 이해할 수 있도록 가능한 깔끔하고 정리된 결과가 나오면 좋습니다.

** 데이터셋 **
* 파일 포맷: TSV
* 파일 용량: 228M (압축해서 26M)
* 샘플 수: 650,965 건
* feature 수: 167 개
  * 1 : label. 해당 유저가 목록에 진입하고 1시간 이내에 구매했는지 여부
  * 2 : 사용 플랫폼 A
  * 3 : 사용 플랫폼 B
  * 4 : 사용 플랫폼 C
  * 5 : 사용 플랫폼 D
  * 6 : 목록 진입시점 방문 총 세션 수 (범위별로 부여된 순차 ID)
  * 7 : 작품을 나타내는 해쉬
  * 8-10 : 개인정보
  * 11-110 : 주요 작품 구매 여부
  * 111 : 작품 태그 정보
  * 112 : 구매할 때 필요한 코인
  * 113 : 완결 여부
  * 114-123 : 스케쥴 정보
  * 124-141 : 장르 정보
  * 142 : 해당 작품의 마지막 에피소드 발행 시점 (범위별로 부여된 순차 ID)
  * 143 : 단행본 여부
  * 144 : 작품 발행 시점 (범위별로 부여된 순차 ID)
  * 145 : 총 발행 에피소드 수 (범위별로 부여된 순차 ID)
  * 146-151 : 작품 태그 정보
  * 152-167 : 유저의 성향 정보 (과거에 구매를 했을 때만 기록)

## 데이터 로딩하기

In [2]:
data = pd.read_csv("../../../Data_Analysis/data/lezhin_dataset_v2_training.tsv", sep="\t", header=None)

print(data.shape)
data.head()

(650965, 167)


,0,1,2,3,4,5,6,7,8,9,...,157,158,159,160,161,162,163,164,165,166
0,0,0,0,0,1,1,001C9D9B,8337B6FB,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.2580,0.0030,NaN,NaN
1,0,0,0,0,1,1,001C9D9B,90D8AB70,1,NaN,...,NaN,0.5000,NaN,NaN,NaN,NaN,0.5000,NaN,NaN,NaN
2,0,0,0,0,1,1,001C9D9B,ABC21E80,1,NaN,...,NaN,0.0187,0.0047,NaN,NaN,NaN,0.0287,0.0055,NaN,NaN
3,0,0,0,0,1,1,001C9D9B,C17967D1,0,69EF2C8F,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0186,NaN,NaN,NaN
4,0,0,0,0,1,1,002B4BDE,AF145784,0,NaN,...,NaN,0.0207,NaN,NaN,NaN,NaN,0.2805,0.0692,NaN,NaN


### 1. 사용 플랫폼 분석
- 마케팅 채널
    - 마케틷 비용 대비 효율 측정

다음의 데이터를 활용하여 insights를 얻고 싶습니다.

  * 1 : label. 해당 유저가 목록에 진입하고 1시간 이내에 구매했는지 여부 (이하 구매 여부)
  * 2 : 사용 플랫폼 A
  * 3 : 사용 플랫폼 B
  * 4 : 사용 플랫폼 C
  * 5 : 사용 플랫폼 D

** 1. 전체 데이터(650,965 건)에서 구매한 사용자 / 구매하지 않은 사용자의 인원 수를 찾아주세요. **

In [3]:
platform = data[[0, 1, 2, 3, 4]].copy()
platform.columns = ["구매 여부", "사용 플랫폼 A", "사용 플랫폼 B", "사용 플랫폼 C", "사용 플랫폼 D"]

print(platform.shape)
platform.head()

(650965, 5)


,구매 여부,사용 플랫폼 A,사용 플랫폼 B,사용 플랫폼 C,사용 플랫폼 D
0,0,0,0,0,1
1,0,0,0,0,1
2,0,0,0,0,1
3,0,0,0,0,1
4,0,0,0,0,1


In [4]:
platform["구매 여부"].value_counts()

0    429172
1    221793
Name: 구매 여부, dtype: int64

** 2. 사용 플랫폼(A~D)별 구매한 사용자 / 구매하지 않은 사용자의 총 인원 수를 찾아주세요. . **

즉, 표의 세로축에는 구매 여부(0, 1)가, 가로축에서는 각 사용 플랫폼(A~D)별 구매한 사용자 / 구매하지 않은 사용자의 총 인원 수가 나왔으면 좋겠습니다.

In [5]:
## 확률
pd.pivot_table(platform,
               index = "구매 여부",
               values=["사용 플랫폼 A", "사용 플랫폼 B", "사용 플랫폼 C", "사용 플랫폼 D"])

,사용 플랫폼 A,사용 플랫폼 B,사용 플랫폼 C,사용 플랫폼 D
구매 여부,,,,
0,0.174021,0.348632,0.146908,0.330439
1,0.068118,0.192662,0.202996,0.536225


In [6]:
## 합계
pd.pivot_table(platform,
               index = "구매 여부",
               values=["사용 플랫폼 A", "사용 플랫폼 B", "사용 플랫폼 C", "사용 플랫폼 D"],
               aggfunc='sum')

,사용 플랫폼 A,사용 플랫폼 B,사용 플랫폼 C,사용 플랫폼 D
구매 여부,,,,
0,74685,149623,63049,141815
1,15108,42731,45023,118931


** 3. 2번 데이터에서 총 인원수는 물론, 평균 구매율도 함께 계산해주세요.**

힌트: 총 인원 수를 sum이라고 한다면, 평균 구매율은 mean이라고 할 수 있습니다.



In [7]:
## 합계 & 평균
import numpy as np

pd.pivot_table(platform,
               index = "구매 여부",
               values=["사용 플랫폼 A", "사용 플랫폼 B", "사용 플랫폼 C", "사용 플랫폼 D"],
               aggfunc=[np.sum, np.mean])

sum                                 mean                      \
      사용 플랫폼 A 사용 플랫폼 B 사용 플랫폼 C 사용 플랫폼 D  사용 플랫폼 A  사용 플랫폼 B  사용 플랫폼 C   
구매 여부                                                                     
0        74685   149623    63049   141815  0.174021  0.348632  0.146908   
1        15108    42731    45023   118931  0.068118  0.192662  0.202996   

                 
       사용 플랫폼 D  
구매 여부            
0      0.330439  
1      0.536225

** 4. '사용 플랫폼' 이라는 새로운 컬럼을 만들어주세요 **

데이터를 분석한 결과, 두 개 이상의 플랫폼에서 동시에 방문하는 경우는 찾을 수 없었습니다.

그러므로 사용 플랫폼 A, 사용 플랫폼 B, 사용 플랫폼 C, 사용 플랫폼 D를 따로 나누지 않고, **사용 플랫폼**이라는 하나의 컬럼으로 합쳤으면 좋겠습니다. 해당 컬럼에는 A, B, C, D 값 중 하나가 들어갑니다.


In [8]:
platform.head()

,구매 여부,사용 플랫폼 A,사용 플랫폼 B,사용 플랫폼 C,사용 플랫폼 D
0,0,0,0,0,1
1,0,0,0,0,1
2,0,0,0,0,1
3,0,0,0,0,1
4,0,0,0,0,1


In [9]:
platform.loc[platform["사용 플랫폼 A"] == 1, "사용 플랫폼"] = "A"
platform.loc[platform["사용 플랫폼 B"] == 1, "사용 플랫폼"] = "B"
platform.loc[platform["사용 플랫폼 C"] == 1, "사용 플랫폼"] = "C"
platform.loc[platform["사용 플랫폼 D"] == 1, "사용 플랫폼"] = "D"

print(platform.shape)
platform.head()

(650965, 6)


,구매 여부,사용 플랫폼 A,사용 플랫폼 B,사용 플랫폼 C,사용 플랫폼 D,사용 플랫폼
0,0,0,0,0,1,D
1,0,0,0,0,1,D
2,0,0,0,0,1,D
3,0,0,0,0,1,D
4,0,0,0,0,1,D


### 2. 작품 분석

다음의 데이터를 활용하여 insights를 얻고 싶습니다.

  * 1 : label. 해당 유저가 목록에 진입하고 1시간 이내에 구매했는지 여부 (이하 구매 여부)
  * 2 : 사용 플랫폼 A
  * 3 : 사용 플랫폼 B
  * 4 : 사용 플랫폼 C
  * 5 : 사용 플랫폼 D
  * 7 : 작품을 나타내는 해쉬 (이하 작품 번호)
  * 112 : 구매할 때 필요한 코인 (이하 구매 코인)
 

여기서 코인 1개의 가격은 100원이라고 가정합니다.

In [83]:
comics = data[[0, 1, 2, 3, 4, 6, 111]].copy()

comics.columns = ["구매 여부", "사용 플랫폼 A", "사용 플랫폼 B", "사용 플랫폼 C", "사용 플랫폼 D", "작품 번호", "구매 코인"]

print(comics.shape)
comics.head()

(650965, 7)


,구매 여부,사용 플랫폼 A,사용 플랫폼 B,사용 플랫폼 C,사용 플랫폼 D,작품 번호,구매 코인
0,0,0,0,0,1,001C9D9B,4
1,0,0,0,0,1,001C9D9B,4
2,0,0,0,0,1,001C9D9B,4
3,0,0,0,0,1,001C9D9B,4
4,0,0,0,0,1,002B4BDE,3


In [84]:
# 5
comics["작품 번호"].unique().shape

(3559,)

In [85]:
# 6
purchased_comics = comics[comics["구매 여부"] == 1]
purchased_comics["작품 번호"].head(10)

216289    001C9D9B
216290    002B4BDE
216291    00A49090
216292    00A49090
216293    00A49090
216294    00A49090
216295    00A49090
216296    00A49090
216297    00A49090
216298    00A49090
Name: 작품 번호, dtype: object

In [87]:
# 8

comics["가격"]  = comics["구매 코인"] * 100
comics.loc[comics["구매 여부"] == 0, "가격"] = 0

In [ ]:
sales = comics

In [ ]:
# 9

In [89]:
# 10

top_comics = comics["작품 번호"].value_counts()
top_comics = top_comics[top_comics >= 100]
top_comics

6983846D    9690
00A49090    8782
0365FD34    8443
621BF66D    7665
D619AC7D    7380
0757A410    6644
37C9A863    6487
85803B60    6083
AC627AB1    5667
D8CD8806    5468
B74E1909    5334
A7D44D4B    5022
E1AE71FB    4839
EC612BAF    4743
46F57FFA    4727
F0A27F22    4677
4847D13D    4673
BEE0E409    4519
330C9DF7    4515
707F7107    4487
CFDECE02    4459
69F29D71    4424
5977BD95    4392
B3298042    4115
B0A70AC1    4006
A01A0380    3979
7394AF9E    3950
6E6A8431    3921
607C8CCE    3716
81FD3004    3699
            ... 
35862DB2     118
CB389B23     117
57F87613     117
C510C556     115
C3C59E5F     115
C6835303     112
4BAC9327     112
344E7C9D     109
DC912A25     109
67C6A1E7     109
B4288D9C     108
3BDA4C72     108
8BD2D693     108
D803C3A2     108
FC6FC140     107
DB8E1AF0     107
7860818E     106
D1FE173D     105
D04A3C8F     105
854D6FAE     105
C21EDEEA     104
C399EE1A     104
1AA27FD7     102
0E9821BF     102
33E16A43     102
C45147DE     102
9CFDF10E     102
AC1DD209     1

** 5. 총 작품의 갯수를 구해주세요. **

In [10]:
platform_insights = data[[0, 1, 2, 3, 4, 6, 111]].copy()
platform_insights.columns = ["구매 여부", "사용 플랫폼 A", "사용 플랫폼 B", "사용 플랫폼 C", "사용 플랫폼 D", "작품 번호", "구매 코인"]

print(platform_insights.shape)
platform_insights.head()

(650965, 7)


,구매 여부,사용 플랫폼 A,사용 플랫폼 B,사용 플랫폼 C,사용 플랫폼 D,작품 번호,구매 코인
0,0,0,0,0,1,001C9D9B,4
1,0,0,0,0,1,001C9D9B,4
2,0,0,0,0,1,001C9D9B,4
3,0,0,0,0,1,001C9D9B,4
4,0,0,0,0,1,002B4BDE,3


In [25]:
group_category = platform_insights.groupby("작품 번호")

group_category.describe()

구매 여부                                                  구매 코인       \
           count      mean       std  min   25%  50%   75%  max   count mean   
작품 번호                                                                          
001C9D9B   272.0  0.327206  0.470058  0.0  0.00  0.0  1.00  1.0   272.0  4.0   
002B4BDE   580.0  0.382759  0.486480  0.0  0.00  0.0  1.00  1.0   580.0  3.0   
0030E221     5.0  0.600000  0.547723  0.0  0.00  1.0  1.00  1.0     5.0  2.0   
00343701     4.0  0.500000  0.577350  0.0  0.00  0.5  1.00  1.0     4.0  0.0   
003C31DA     3.0  0.000000  0.000000  0.0  0.00  0.0  0.00  0.0     3.0  2.0   
004259EF     1.0  0.000000       NaN  0.0  0.00  0.0  0.00  0.0     1.0  0.0   
004D740D     1.0  0.000000       NaN  0.0  0.00  0.0  0.00  0.0     1.0  4.0   
00545B4F   127.0  0.251969  0.435863  0.0  0.00  0.0  0.50  1.0   127.0  4.0   
00548AD2     2.0  0.000000  0.000000  0.0  0.00  0.0  0.00  0.0     2.0  4.0   
00563F3B     1.0  1.000000       NaN  1.0  1.00  1.0  1.00  1.0     1.0  4.0   
0071D229     1.0  1.000000       NaN  1.0  1.00  1.0  1.00  1.0     1.0  4.0   
0072CE25     2.0  0.000000  0.000000  0.0  0.00  0.0  0.00  0.0     2.0  0.0   
00A49090  8782.0  0.603279  0.489245  0.0  0.00  1.0  1.00  1.0  8782.0  3.0   
00A5099D    44.0  0.272727  0.450511  0.0  0.00  0.0  1.00  1.0    44.0  0.0   
00AC8ED3     1.0  0.000000       NaN  0.0  0.00  0.0  0.00  0.0     1.0  0.0   
00E3ED9E     7.0  0.000000  0.000000  0.0  0.00  0.0  0.00  0.0     7.0  3.0   
00EC53C4   136.0  0.536765  0.500490  0.0  0.00  1.0  1.00  1.0   136.0  0.0   
01161AAA     1.0  0.000000       NaN  0.0  0.00  0.0  0.00  0.0     1.0  0.0   
011A000A    10.0  0.400000  0.516398  0.0  0.00  0.0  1.00  1.0    10.0  4.0   
012831C2    48.0  0.270833  0.449093  0.0  0.00  0.0  1.00  1.0    48.0  4.0   
01386BD6  2780.0  0.413309  0.492516  0.0  0.00  0.0  1.00  1.0  2780.0  0.0   
013D0A4E    20.0  0.500000  0.512989  0.0  0.00  0.5  1.00  1.0    20.0  4.0   
013D4071     3.0  0.000000  0.000000  0.0  0.00  0.0  0.00  0.0     3.0  0.0   
014B2D6C     1.0  0.000000       NaN  0.0  0.00  0.0  0.00  0.0     1.0  4.0   
015C30F4    44.0  0.477273  0.505258  0.0  0.00  0.0  1.00  1.0    44.0  3.0   
0160947B    11.0  0.090909  0.301511  0.0  0.00  0.0  0.00  1.0    11.0  2.0   
01712C18     2.0  0.500000  0.707107  0.0  0.25  0.5  0.75  1.0     2.0  3.0   
0177A314     1.0  0.000000       NaN  0.0  0.00  0.0  0.00  0.0     1.0  0.0   
01BA9398   299.0  0.324415  0.468941  0.0  0.00  0.0  1.00  1.0   299.0  3.0   
01F78BE6   128.0  0.273438  0.447475  0.0  0.00  0.0  1.00  1.0   128.0  0.0   
...          ...       ...       ...  ...   ...  ...   ...  ...     ...  ...   
FE3533ED     1.0  0.000000       NaN  0.0  0.00  0.0  0.00  0.0     1.0  0.0   
FE3DF164    91.0  0.252747  0.436995  0.0  0.00  0.0  0.50  1.0    91.0  4.0   
FE3E04FB     2.0  0.500000  0.707107  0.0  0.25  0.5  0.75  1.0     2.0  3.0   
FE44EE02     2.0  0.000000  0.000000  0.0  0.00  0.0  0.00  0.0     2.0  2.0   
FE63D198     3.0  0.000000  0.000000  0.0  0.00  0.0  0.00  0.0     3.0  0.0   
FE6CDF91     9.0  0.111111  0.333333  0.0  0.00  0.0  0.00  1.0     9.0  2.0   
FE7EE8FC     1.0  0.000000       NaN  0.0  0.00  0.0  0.00  0.0     1.0  0.0   
FE82FA3D     4.0  0.000000  0.000000  0.0  0.00  0.0  0.00  0.0     4.0  2.0   
FE9E25D2     5.0  0.000000  0.000000  0.0  0.00  0.0  0.00  0.0     5.0  0.0   
FE9FC289     2.0  0.000000  0.000000  0.0  0.00  0.0  0.00  0.0     2.0  0.0   
FEB2AD6A     4.0  0.000000  0.000000  0.0  0.00  0.0  0.00  0.0     4.0  4.0   
FED33392     1.0  0.000000       NaN  0.0  0.00  0.0  0.00  0.0     1.0  0.0   
FEE17FB8     1.0  0.000000       NaN  0.0  0.00  0.0  0.00  0.0     1.0  4.0   
FF1E4832     1.0  0.000000       NaN  0.0  0.00  0.0  0.00  0.0     1.0  2.0   
FF31C718    12.0  0.583333  0.514929  0.0  0.00  1.0  1.00  1.0    12.0  2.0   
FF455F4A    31.0  0.129032  0.340777  0.0  0.00  0.0  0.00  1.0    31.0  3.0   
FF4D5FBB   

In [28]:
group_category["작품 번호"].count().sort_values(ascending=False)

작품 번호
6983846D    9690
00A49090    8782
0365FD34    8443
621BF66D    7665
D619AC7D    7380
0757A410    6644
37C9A863    6487
85803B60    6083
AC627AB1    5667
D8CD8806    5468
B74E1909    5334
A7D44D4B    5022
E1AE71FB    4839
EC612BAF    4743
46F57FFA    4727
F0A27F22    4677
4847D13D    4673
BEE0E409    4519
330C9DF7    4515
707F7107    4487
CFDECE02    4459
69F29D71    4424
5977BD95    4392
B3298042    4115
B0A70AC1    4006
A01A0380    3979
7394AF9E    3950
6E6A8431    3921
607C8CCE    3716
81FD3004    3699
            ... 
D3595D5A       1
D34FF294       1
D4C2E4A3       1
4AA32C2F       1
D3232F2E       1
D2ED45A5       1
D2DDEA18       1
4B025079       1
D240E3D3       1
92C8C96E       1
D392D390       1
48F1FA27       1
48F008FC       1
48E92E6A       1
D498CFD5       1
4753CF5C       1
93DB85ED       1
47C41C8A       1
47D1E990       1
47F94506       1
480CFD6F       1
AEF95017       1
4836897E       1
D3FA90FA       1
4867EB8B       1
487BD954       1
D3E13478       1
48DC8047

In [11]:
platform_insights["작품 번호"].value_counts()

6983846D    9690
00A49090    8782
0365FD34    8443
621BF66D    7665
D619AC7D    7380
0757A410    6644
37C9A863    6487
85803B60    6083
AC627AB1    5667
D8CD8806    5468
B74E1909    5334
A7D44D4B    5022
E1AE71FB    4839
EC612BAF    4743
46F57FFA    4727
F0A27F22    4677
4847D13D    4673
BEE0E409    4519
330C9DF7    4515
707F7107    4487
CFDECE02    4459
69F29D71    4424
5977BD95    4392
B3298042    4115
B0A70AC1    4006
A01A0380    3979
7394AF9E    3950
6E6A8431    3921
607C8CCE    3716
81FD3004    3699
            ... 
54BD38C9       1
A9393B1E       1
641F6F1D       1
2ED7920A       1
7F20DB15       1
098B3FFE       1
2E68898B       1
DCE0D391       1
1037285A       1
818F4654       1
702123F9       1
F6F15A79       1
12D67DB4       1
4836897E       1
E207EA1F       1
7F42F86E       1
1AA921C5       1
870B1CDF       1
F39BE7DD       1
66A78A3A       1
8E728D21       1
D8B7AC65       1
2D26C9BB       1
9D61A998       1
C058F544       1
90AE5F9B       1
3455F3CB       1
738901DB      

In [74]:
len(platform_insights.groupby('작품 번호').nunique())

3559

** 6. 가장 많이 구매한 작품 Top 10을 찾아주세요. **

In [13]:
platform_insights[["작품 번호","구매 코인"]].sort_values(by="구매 코인", ascending=False).head(10)

,작품 번호,구매 코인
77443,CC6D7796,50
57529,BB4F4756,50
139154,CC6D7796,50
448017,CC6D7796,50
26058,CC6D7796,50
532168,1DF2BF5B,50
391624,CC6D7796,50
175811,CC6D7796,50
577079,CC6D7796,50
577068,CC6D7796,50


In [29]:
df = platform_insights.groupby('작품 번호').nunique()
print(df)

          구매 여부  사용 플랫폼 A  사용 플랫폼 B  사용 플랫폼 C  사용 플랫폼 D  작품 번호  구매 코인  매출
작품 번호                                                                    
001C9D9B      2         1         2         2         2      1      1   2
002B4BDE      2         1         2         2         2      1      1   2
0030E221      2         1         1         2         2      1      1   1
00343701      2         1         1         2         2      1      1   1
003C31DA      1         1         1         2         2      1      1   1
004259EF      1         1         1         1         1      1      1   1
004D740D      1         1         1         1         1      1      1   1
00545B4F      2         1         1         2         2      1      1   1
00548AD2      1         1         1         1         1      1      1   1
00563F3B      1         1         1         1         1      1      1   1
0071D229      1         1         1         1         1      1      1   1
0072CE25      1         1         1   

In [40]:
df["구매 여부"].sort_values(ascending=False).head(10)

작품 번호
82161242    2
75DDC47C    2
76DC815A    2
76DC611D    2
766EBCD5    2
764833C8    2
7647966B    2
75FAB6A9    2
75CD5A46    2
79CA1044    2
Name: 구매 여부, dtype: int64

** 7. 정 반대로, 가장 적게 구매한 작품 Top10을 구해주세요. **

In [14]:
platform_insights[["작품 번호","구매 코인"]].sort_values(by="구매 코인", ascending=True).head(10)

,작품 번호,구매 코인
370852,246A40F6,0
99357,C1F9B722,0
99353,B8B1D1B3,0
412861,6C4B761A,0
99347,A9961AFB,0
412855,59B90E10,0
99343,A60F1D36,0
316346,E369853D,0
99340,9F61408E,0
99359,C51CE410,0


In [39]:
df["구매 여부"].sort_values().head(10)

작품 번호
FFF1FD1F    1
71A3CB15    1
719BE142    1
717976BA    1
715D0635    1
7154A595    1
7143D7FB    1
713026BE    1
71277BEF    1
70C5C25F    1
Name: 구매 여부, dtype: int64

** 8. 구매 코인과 코인 가격을 고려하여, 가장 높은 매출을 달성한 작품 Top 10을 구해주세요. **

한 번 구매했을 때 가격은 구매 코인 * 가격(100 원)이라고 볼 수 있습니다. 가령 구매 코인이 4라면 가격은 400 원이 됩니다.

In [15]:
platform_insights["구매 코인"] * 100

0         400
1         400
2         400
3         400
4         300
5         300
6         300
7         300
8         300
9         300
10        300
11        300
12        300
13        300
14        300
15        300
16        300
17        300
18          0
19          0
20          0
21          0
22        300
23        300
24        300
25          0
26          0
27          0
28        300
29          0
         ... 
650935    300
650936    300
650937    300
650938    300
650939    300
650940    300
650941    300
650942    300
650943    300
650944    300
650945    200
650946    300
650947    300
650948    300
650949    300
650950    300
650951      0
650952      0
650953      0
650954    200
650955    200
650956    200
650957    200
650958    300
650959    300
650960    200
650961    200
650962    300
650963    300
650964    300
Name: 구매 코인, Length: 650965, dtype: int64

In [59]:
# platform_insights.loc[platform_insights["구매 코인"], "매출"] = platform_insights["구매 코인"] * 100

In [57]:
# df[["구매 여부", "구매 코인"]].sort_values(by="구매 여부",ascending=False)

In [44]:
df2 = df[["구매 여부", "구매 코인"]].sort_values(by="구매 여부",ascending=False)
df2

,구매 여부,구매 코인
작품 번호,,
001C9D9B,2,1
8ED81745,2,1
8DF707A9,2,1
8E1CD19E,2,1
8E296A06,2,1
8E874A1D,2,1
8E98D81F,2,1
8E9C97EB,2,1
8EE144B1,2,1


In [56]:
# df2["매출"] = df2[df2["구매 여부"] * df["구매 코인"] * 100]

In [51]:
df3 = df2["구매 여부"] * df["구매 코인"] * 100
df3

작품 번호
001C9D9B    200
002B4BDE    200
0030E221    200
00343701    200
003C31DA    100
004259EF    100
004D740D    100
00545B4F    200
00548AD2    100
00563F3B    100
0071D229    100
0072CE25    100
00A49090    200
00A5099D    200
00AC8ED3    100
00E3ED9E    100
00EC53C4    200
01161AAA    100
011A000A    200
012831C2    200
01386BD6    200
013D0A4E    200
013D4071    100
014B2D6C    100
015C30F4    200
0160947B    200
01712C18    200
0177A314    100
01BA9398    200
01F78BE6    200
           ... 
FE3533ED    100
FE3DF164    200
FE3E04FB    200
FE44EE02    100
FE63D198    100
FE6CDF91    200
FE7EE8FC    100
FE82FA3D    100
FE9E25D2    100
FE9FC289    100
FEB2AD6A    100
FED33392    100
FEE17FB8    100
FF1E4832    100
FF31C718    200
FF455F4A    200
FF4D5FBB    100
FF5DBAC3    100
FF6C9FC2    100
FF6D89C3    200
FF7259D4    100
FF77B8FC    200
FFA39014    200
FFB6A968    200
FFB6E9B2    100
FFC5BE2E    100
FFD52F3C    200
FFE303DC    100
FFEABD22    200
FFF1FD1F    100
Length: 3559, dtyp

In [55]:
df3.sort_values(ascending=False).head(10)

작품 번호
82161242    200
75DDC47C    200
76DC815A    200
76DC611D    200
766EBCD5    200
764833C8    200
7647966B    200
75FAB6A9    200
75CD5A46    200
79CA1044    200
dtype: int64

** 9. 몇몇 주요 작품들의 사용 플랫폼별 구매 횟수를 구해주세요 **

다음의 다섯 개 아이디(**D91D1B4D**, **72B32A1F**, **06138BC5**, **7F7DDD7D**, **DBB99DB2**)를 주요 작품으로 간주합니다.

해당 다섯 개의 작품이 사용 플랫폼 A~D에서 구매한 횟수를 각각 구해주세요.

In [76]:
platform_insights[platform_insights["구매 여부"] == 1]

,구매 여부,사용 플랫폼 A,사용 플랫폼 B,사용 플랫폼 C,사용 플랫폼 D,작품 번호,구매 코인,매출
216289,1,0,0,0,1,001C9D9B,4,-9223372036854775808
216290,1,0,0,0,1,002B4BDE,3,-9223372036854775808
216291,1,0,0,0,1,00A49090,3,-9223372036854775808
216292,1,0,0,0,1,00A49090,3,-9223372036854775808
216293,1,0,0,0,1,00A49090,3,-9223372036854775808
216294,1,0,0,0,1,00A49090,3,-9223372036854775808
216295,1,0,0,0,1,00A49090,3,-9223372036854775808
216296,1,0,0,0,1,00A49090,3,-9223372036854775808
216297,1,0,0,0,1,00A49090,3,-9223372036854775808
216298,1,0,0,0,1,00A49090,3,-9223372036854775808


In [81]:
platform_insights.loc[platform_insights["구매 여부"] == 1 & platform_insights["사용 플랫폼 A"] == 1, "구매 플랫폼"] = "A"
platform_insights.loc[platform_insights["구매 여부"] == 1 & platform_insights["사용 플랫폼 B"] == 1, "구매 플랫폼"] = "B"
platform_insights.loc[platform_insights["구매 여부"] == 1 & platform_insights["사용 플랫폼 C"] == 1, "구매 플랫폼"] = "C"
platform_insights.loc[platform_insights["구매 여부"] == 1 & platform_insights["사용 플랫폼 D"] == 1, "구매 플랫폼"] = "D"

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
print(platform_insights.shape)
platform_insights.head()

** 10. 100회 이상 구매한 작품 중, 가장 구매 확률이 높은 작품 top 10을 찾아주세요. **

팁: 세 개를 동시에 시도하지 않고, 1) 100회 이상 구매하지 않은 작품을 제거한 뒤, 2) 1번에서 작품당 구매확률을 구하고, 3) 2번에서 구매 확률로 정렬하는 방식으로 일을 쪼개서 작업하면 조금 더 쉽게 구현할 수 있습니다.

In [19]:
# Write your code here!

### 3. 주요 작품 분석

다음의 데이터를 활용하여 insights를 얻고 싶습니다.

  * 1 : label. 해당 유저가 목록에 진입하고 1시간 이내에 구매했는지 여부 (이하 구매 여부)
  * 7 : 작품을 나타내는 해쉬 (이하 작품 번호)
  * 11-110 : 주요 작품 구매 여부. (이하 주요 작품 1 ~ 주요 작품 100)

** 11. 가장 많은 횟수를 구매한 주요 작품 Top 10을 찾아주세요. **

In [20]:
# Write your code here!

** 12. 구매 여부를 기준으로, 구매하지 않은 사용자는 배제하고 구매한 사용자 기준으로 가장 많은 횟수를 구매한 주요 작품 Top 10을 찾아주세요. **

In [21]:
# Write your code here!

** 13. 구매 횟수와 상관 없이, 구매 확률이 가장 높은 주요 작품 Top 10을 찾아주세요. **

가령 구매를 20번 하건 30번 하건 전부 1번으로 가정 합니다. 또 다른 예를 들자면, 전체 650,965 개의 데이터에서 주요 작품 1번을 6,509번 구매했다면 구매 확률은 0.01(1%)로 가정합니다.

In [22]:
# Write your code here!

** 14. 작품 번호별로, 구매 횟수가 가장 높은 주요 작품(주요 작품 1 ~ 100)의 Top 10을 찾아주세요. **

또한 작품 번호는 누적 구매 수가 가장 많은 순으로 정렬, 주요 작품도 누적 구매 수가 가장 많은 순으로 정렬합니다.

In [23]:
# Write your code here!

In [24]:
df = platform_insights.groupby('작품 번호').nunique()
print(df)

          구매 여부  사용 플랫폼 A  사용 플랫폼 B  사용 플랫폼 C  사용 플랫폼 D  작품 번호  구매 코인  매출
작품 번호                                                                    
001C9D9B      2         1         2         2         2      1      1   2
002B4BDE      2         1         2         2         2      1      1   2
0030E221      2         1         1         2         2      1      1   1
00343701      2         1         1         2         2      1      1   1
003C31DA      1         1         1         2         2      1      1   1
004259EF      1         1         1         1         1      1      1   1
004D740D      1         1         1         1         1      1      1   1
00545B4F      2         1         1         2         2      1      1   1
00548AD2      1         1         1         1         1      1      1   1
00563F3B      1         1         1         1         1      1      1   1
0071D229      1         1         1         1         1      1      1   1
0072CE25      1         1         1   